<a href="https://colab.research.google.com/github/rayarxti/QuanTemp/blob/main/eval_oracle_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

from transformers import AutoTokenizer,AutoModel

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli', do_lower_case=True)

Loading BERT tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import json
with open("/content/drive/MyDrive/Agrawal Lab/QuanTemp/data/raw_data/test_claims_quantemp.json") as f:
  test_data = json.load(f)

In [ ]:
test_data[-1]

{'crawled_date': '2022-09-24',
 'country_of_origin': 'india',
 'label': 'False',
 'url': 'https://www.indiatoday.in/fact-check/story/fact-check-woman-seen-with-rahul-gandhi-is-not-amulya-leona-student-who-chanted-pak-zindabad-2004338-2022-09-24',
 'lang': 'en',
 'claim': "During the Bharat Jodo Yatra, Rahul Gandhi was seen with Amulya Leona Noronha, the woman who was arrested for saying 'Pakistan Zindabad' during an anti-CAA rally in 2020.",
 'doc': 'AFWA\'s investigation found that the woman seen in the viral photo is not Amulya Leona Noronha. Rahul Gandhi and members of the Congress party, as part of the Bharat Jodo Yatra, have met locals and leaders alike as they move from city to city on foot. Of the many, many photos from the road, one shows Gandhi posing with a young woman. The Congress leader was giving her a side-hug. This photo has been widely shared with some questionable claims. They alleged that the woman in the photo is Amulya Leona Noronha, a student who chanted “Pakistan

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

In [ ]:
def get_features(data):
  features = []
  evidences = []

  for index, fact in enumerate(data):
    claim = fact["claim"]


    feature = "[Claim]:"+claim+"[Evidences]:"+fact["doc"]
    features.append(feature)
  return features

test_features = get_features(test_data)

In [ ]:
with open("/content/drive/MyDrive/Agrawal Lab/QuanTemp/data/raw_data/train_claims_quantemp.json") as f:
  train_data = json.load(f)
train_labels = [fact["label"] for fact in train_data]
train_labels_final = LE.fit_transform(train_labels)

test_labels = [fact["label"] for fact in test_data]
test_labels_final = LE.transform(test_labels)
test_labels_final

array([0, 2, 1, ..., 1, 1, 1])

In [ ]:
input_ids = []
attention_masks = []

for sent in test_features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', test_features[0])
print('Token IDs:', input_ids[0])

test_labels_final = torch.tensor(test_labels_final)

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  [Claim]:"The non-partisan Congressional Budget Office concluded ObamaCare will cost the U.S. more than 800,000 jobs."[Evidences]:Republican U.S House candidate Roger Williams of Texas promises to "end the fraud of Obamacare" in a mailer to Austin-area voters we spotted April 11, 2012.His mailer lists among promises broken by President Obama the notion that the 2010 overhaul of health care laws would create jobs. Under "Promise Broken," the mailer says: "The non-partisan Congressional Budget Office concluded ObamaCare will cost the U.S. more than 800,000 jobs."Really?Williams’ footnoted backup was a Feb. 11, 2011, blog post on AmericanThinker.com saying that Doug Elmendorf, director of the Congressional Budget Office, had just testified to the House Budget Committee that as of 2020-21, the law was projected to reduce U.S. employment by 800,000.PolitiFact researchers have explored this testimony and topic before, most recently for a June 2011 Truth-O-Meter article after U.S. R

In [ ]:
from torch.utils.data import TensorDataset, random_split
# train_poincare_tensor = torch.tensor(poincare_embeddings_final,dtype=torch.float)
# difficulty_tensor = torch.tensor(difficulty_level_vectors,dtype=torch.float)
# Combine the training inputs into a TensorDataset.
test_dataset = TensorDataset(input_ids, attention_masks, test_labels_final)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 16
test_dataloader = DataLoader(
    test_dataset, # The test samples.
    sampler = SequentialSampler(test_dataset),
    batch_size = batch_size
)

In [ ]:
from torch import nn
class MultiClassClassifier(nn.Module):
    def __init__(self, bert_model_path, labels_count, hidden_dim=768, mlp_dim=500, extras_dim=100, dropout=0.1, freeze_bert=False):
        super().__init__()

        self.roberta = AutoModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=True)
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_dim),
            nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # # nn.ReLU(),
            # # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),
            nn.Linear(mlp_dim, labels_count)
        )
        # self.softmax = nn.LogSoftmax(dim=1)
        if freeze_bert:
            print("Freezing layers")
            for param in self.roberta.parameters():
                param.requires_grad = False

    def forward(self, tokens, masks):
        output = self.roberta(tokens, attention_mask=masks)
        dropout_output = self.dropout(output["pooler_output"])
        # concat_output = torch.cat((dropout_output, topic_emb), dim=1)
        # concat_output = self.dropout(concat_output)
        mlp_output = self.mlp(dropout_output)
        # proba = self.sigmoid(mlp_output)
        # proba = self.softmax(mlp_output)

        return mlp_output

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Loads BertForSequenceClassification, the pretrained BERT model with a single
model = MultiClassClassifier('roberta-large-mnli',3, 1024,768,140,dropout=0.1,freeze_bert=False)

# model.load_state_dict(torch.load("model_bert_difficulty_prediction/model_weights"))

# Tell pytorch to run this model on the GPU.
model.load_state_dict(torch.load("/content/drive/MyDrive/Agrawal Lab/QuanTemp/model_roberta_large_oracle/model_weights", weights_only=True))

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

<All keys matched successfully>

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
loss_func = nn.CrossEntropyLoss()

In [ ]:
test_preds = np.zeros((0,))
test_preds_prob = np.zeros((0, 3))

In [ ]:
model = model.to(device)

# ========================================
#               Validation
# ========================================
# After the completion of each training epoch, measure our performance on
# our validation set.

print("")
print("Running test...")

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

# Evaluate data for one epoch
for batch in test_dataloader:

    # Unpack this training batch from our dataloader.
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids
    #   [1]: attention masks
    #   [2]: labels
    b_input_ids = batch[0].to(device)

    b_input_mask = batch[1].to(device)
    # b_poincare = batch[2].to(device)
    # b_difficulty = batch[3].to(device)
    b_labels = batch[2].to(device)
    # skill_labels = batch[3].to(device)

    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():

        # Forward pass, calculate logit predictions.

        logits = model(b_input_ids,b_input_mask)

        # Accumulate the validation loss.
        loss = loss_func(logits, b_labels)
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        test_preds_prob = np.concatenate((test_preds_prob, logits))
        test_preds = np.concatenate((test_preds, np.argmax(logits, axis=1).flatten()))


# Report the final accuracy for this validation run.
avg_test_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_test_accuracy))

# Calculate the average loss over all of the batches.
avg_test_loss = total_eval_loss / len(test_dataloader)

print("  Test Loss: {0:.2f}".format(avg_test_loss))


Running test...


RobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


  Accuracy: 0.69
  Test Loss: 0.71


In [ ]:
test_preds = test_preds.astype(int)

In [ ]:
import scipy

test_preds_prob = scipy.special.softmax(test_preds_prob, axis=1)
test_preds_prob

array([[3.08623338e-01, 4.50995111e-01, 2.40381552e-01],
       [1.76666368e-01, 4.71709475e-02, 7.76162685e-01],
       [1.16816241e-02, 9.87782478e-01, 5.35897972e-04],
       ...,
       [1.62906696e-01, 8.29086868e-01, 8.00643567e-03],
       [2.03592336e-01, 1.47207472e-01, 6.49200192e-01],
       [2.15329965e-01, 7.82264277e-01, 2.40575823e-03]])

In [ ]:
import pandas as pd

results = pd.DataFrame(data={
    'features': test_features,
    'true_label': test_labels,
    'pred_label': LE.inverse_transform(test_preds.astype(int)),
    'prob_0': test_preds_prob[:, 0],
    'prob_1': test_preds_prob[:, 1],
    'prob_2': test_preds_prob[:, 2],
    'prob_true': test_preds_prob[np.arange(len(test_preds_prob)), test_labels_final]
})

results.to_csv('/content/drive/MyDrive/Agrawal Lab/QuanTemp/model_oracle_roberta_test_results.csv')